<a href="https://colab.research.google.com/github/DEBMALYASEN/ABC/blob/master/bertQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
     |████████████████████████████████| 6.5 MB 31.2 MB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer


In [3]:
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
coqa.head()

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."


In [4]:
coqa['data']

0       {'source': 'wikipedia', 'id': '3zotghdk5ibi9ce...
1       {'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7...
2       {'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn...
3       {'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya...
4       {'source': 'gutenberg', 'id': '3urfvvm165iantk...
                              ...                        
7194    {'source': 'gutenberg', 'id': '34j10vatjfyw0ao...
7195    {'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z...
7196    {'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v...
7197    {'source': 'wikipedia', 'id': '3gs6s824sqxty8v...
7198    {'source': 'cnn', 'id': '31qnsg6a5rtt5m7pens7x...
Name: data, Length: 7199, dtype: object

In [5]:
del coqa["version"]


In [6]:
coqa

,data
0,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...
7194,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


In [8]:
coqa['data'][0]

{'answers': [{'input_text': 'It was formally established in 1475',
   'span_end': 179,
   'span_start': 151,
   'span_text': 'Formally established in 1475',
   'turn_id': 1},
  {'input_text': 'research',
   'span_end': 494,
   'span_start': 454,
   'span_text': 'he Vatican Library is a research library',
   'turn_id': 2},
  {'input_text': 'history, and law',
   'span_end': 511,
   'span_start': 457,
   'span_text': 'Vatican Library is a research library for history, law',
   'turn_id': 3},
  {'input_text': 'philosophy, science and theology',
   'span_end': 545,
   'span_start': 457,
   'span_text': 'Vatican Library is a research library for history, law, philosophy, science and theology',
   'turn_id': 4},
  {'input_text': 'a  project',
   'span_end': 879,
   'span_start': 769,
   'span_text': 'March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts',
   'turn_id': 5},
  {'input_text': 'into periods',
   'span_end': 1127,
   'span_

In [9]:
#required columns in our dataframe
cols = ["text","question","answer"]
#list of lists to create our dataframe
comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)
new_df = pd.DataFrame(comp_list, columns=cols) 
#saving the dataframe to csv file for further loading
new_df.to_csv("CoQA_data.csv", index=False)


In [10]:
new_df

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project
...,...,...,...
108642,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was a sub?,Xabi Alonso
108643,(CNN) -- Cristiano Ronaldo provided the perfec...,Was it his first game this year?,Yes
108644,(CNN) -- Cristiano Ronaldo provided the perfec...,What position did the team reach?,third
108645,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was ahead of them?,Barca.


In [11]:
data=new_df

In [12]:
data

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project
...,...,...,...
108642,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was a sub?,Xabi Alonso
108643,(CNN) -- Cristiano Ronaldo provided the perfec...,Was it his first game this year?,Yes
108644,(CNN) -- Cristiano Ronaldo provided the perfec...,What position did the team reach?,third
108645,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was ahead of them?,Barca.


In [13]:
print("Number of question and answers: ", len(data))

Number of question and answers:  108647


In [14]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [15]:
random_num = np.random.randint(0,len(data))
question = data["question"][random_num]
text = data["text"][random_num]

In [16]:
question

'where did they line ?'

In [17]:
text

'Chapter Sixteen: The Sultan Saladin \n\nIn the third morning Godwin awoke to see the ray of sunrise streaming through the latticed window. \n\nThey fell upon another bed near-by where Wulf still lay sleeping, a bandage on his head that had been hurt in the last charge against the Assassins, and other bandages about his arms and body, which were much bruised in the fight upon the dreadful bridge. \n\nWondrous was it to Godwin to watch him lying there sleeping healthily, notwithstanding his injuries, and to think of what they had gone through together with so little harm; to think, also, of how they had rescued Rosamund out of the very mouth of that earthly hell of which he could see the peaks through the open window-place--out of the very hands of that fiend, its ruler. Reckoning the tale day by day, he reflected on their adventures since they landed at Beirut, and saw how Heaven had guided their every step. \n\nIn face of the warnings that were given them, to visit the Al-je-bal in hi

In [18]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))


The input has a total of 396 tokens.


In [19]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
where      2,073
did        2,106
they       2,027
line       2,240
?          1,029
[SEP]        102
chapter    3,127
sixteen    7,032
:          1,024
the        1,996
sultan     7,544
salad     16,521
##in       2,378
in         1,999
the        1,996
third      2,353
morning    2,851
god        2,643
##win     10,105
awoke     19,179
to         2,000
see        2,156
the        1,996
ray        4,097
of         1,997
sunrise   13,932
streaming  11,058
through    2,083
the        1,996
lattice   17,779
##d        2,094
window     3,332
.          1,012
they       2,027
fell       3,062
upon       2,588
another    2,178
bed        2,793
near       2,379
-          1,011
by         2,011
where      2,073
wu         8,814
##lf      10,270
still      2,145
lay        3,913
sleeping   5,777
,          1,010
a          1,037
bandage   24,446
on         2,006
his        2,010
head       2,132
that       2,008
had        2,018
been       2,042
hurt       3,480
in         1,

In [23]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer:\n{}".format(answer.capitalize()))



In [26]:
text = """New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's famous rhinestone-studded glove from a 1983 performance -- were auctioned off Saturday, reaping a total $2 million. Profits from the auction at the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of only $120,000 in sales. The highly prized memorabilia, which included items spanning the many stages of Jackson's career, came from more than 30 fans, associates and family members, who contacted Julien's Auctions to sell their gifts and mementos of the singer. Jackson's flashy glove was the big-ticket item of the night, fetching $420,000 from a buyer in Hong Kong, China. Jackson wore the glove at a 1983 performance during \"Motown 25,\" an NBC special where he debuted his revolutionary moonwalk. Fellow Motown star Walter \"Clyde\" Orange of the Commodores, who also performed in the special 26 years ago, said he asked for Jackson's autograph at the time, but Jackson gave him the glove instead. "The legacy that [Jackson] left behind is bigger than life for me,\" Orange said. \"I hope that through that glove people can see what he was trying to say in his music and what he said in his music.\" Orange said he plans to give a portion of the proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 percent buyer's premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 20 percent premium."""
question = "Where was the Auction held?"
question_answer(question, text)
#original answer from the dataset
#print("Original answer:\n", data.loc[data["question"] == question]["answer"].values[0]))



Predicted answer:
Hard rock cafe in new york ' s times square


In [28]:
text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")
while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break


Please enter your text: 
British rule in India began in 1757 when, following the British victory at the Battle of Plassey, the English East India Company began exercising control over the country. The East India Company ruled India for 100 years, until it was replaced by direct British rule (often referred to as the British raj) in the wake of the Indian Mutiny in 1857–58. The Indian independence movement began during World War I and was led by Mohandas K. Gandhi, who advocated for a peaceful and nonviolent end to British rule.

Please enter your question: 
British rule began in?

Predicted answer:
1757

Do you want to ask another question based on this text (Y/N)? Y

Please enter your question: 
british ruled for how many years?

Predicted answer:
100 years

Do you want to ask another question based on this text (Y/N)? who is gandhi?

Do you want to ask another question based on this text (Y/N)? Y

Please enter your question: 
who is gandhi?

Predicted answer:
Mohandas k . gandhi

Do 

UnboundLocalError: ignored